In [1]:
!pip install opendatasets -q
!pip install tensorflow
!pip install numpy
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

import opendatasets as od
import json


In [3]:
# download dataset
od.download('https://www.kaggle.com/datasets/erichartford/leetcode-solutions')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

In [2]:

batch_size=64 # Batch size for training.
epochs = 10 # Number of epochs to train for.
latent_dim =256 # Latent dimensionality of the encoding space.
num_samples = 1000 # Number of samples to trian on.

In [3]:
# data_dir='E:\\final_Project\\coders_ml\\leetcode-solutions.jsonl'

# data_set=[]

# # Open the JSONL file
# with open(data_dir, 'r') as file:
#   # Read each line in the file
#   for line in file:
#     # Parse the JSON object in the line
#     data= json.loads(line)
#     data_set.append(data)

data_dir = 'E:\\final_Project\\coders_ml\\leetcode-solutions.jsonl'
data_set = []

# Open the JSONL file with UTF-8 encoding
with open(data_dir, 'r', encoding='utf-8') as file:
    # Read each line in the file
    for line in file:
        # Parse the JSON object in the line
        data = json.loads(line)
        data_set.append(data)


In [4]:
data_set[1]
for key, value in data_set[1].items():
  print(key)

id
slug
title
difficulty
content
answer


In [5]:
data_set[1]['answer']

{'c++': '```cpp\nstruct ListNode {\n    int val;\n    ListNode *next;\n    ListNode(int x) : val(x), next(nullptr) {}\n};\n\nListNode* addTwoNumbers(ListNode* l1, ListNode* l2) {\n    ListNode dummy(0);\n    ListNode* current = &dummy;\n    int carry = 0;\n\n    while (l1 || l2 || carry) {\n        int sum = (l1 ? l1->val : 0) + (l2 ? l2->val : 0) + carry;\n        carry = sum / 10;\n        current->next = new ListNode(sum % 10);\n        current = current->next;\n\n        if (l1) l1 = l1->next;\n        if (l2) l2 = l2->next;\n    }\n\n    return dummy.next;\n}\n```\n',
 'java': '```java\npublic class ListNode {\n    int val;\n    ListNode next;\n    ListNode(int x) { val = x; }\n}\n\npublic ListNode addTwoNumbers(ListNode l1, ListNode l2) {\n    ListNode dummy = new ListNode(0);\n    ListNode current = dummy;\n    int carry = 0;\n\n    while (l1 != null || l2 != null || carry != 0) {\n        int sum = (l1 != null ? l1.val : 0) + (l2 != null ? l2.val : 0) + carry;\n        carry = 

In [6]:
for key, value in data_set[1]['answer'].items():
  print(key)

c++
java
python
javascript
explanation


In [7]:
input_texts=[]
for value in data_set:
  input_texts.append(value['answer']['java'].strip()[7:-3])

print(input_texts[1])


public class ListNode {
    int val;
    ListNode next;
    ListNode(int x) { val = x; }
}

public ListNode addTwoNumbers(ListNode l1, ListNode l2) {
    ListNode dummy = new ListNode(0);
    ListNode current = dummy;
    int carry = 0;

    while (l1 != null || l2 != null || carry != 0) {
        int sum = (l1 != null ? l1.val : 0) + (l2 != null ? l2.val : 0) + carry;
        carry = sum / 10;
        current.next = new ListNode(sum % 10);
        current = current.next;

        if (l1 != null) l1 = l1.next;
        if (l2 != null) l2 = l2.next;
    }

    return dummy.next;
}



In [8]:
target_texts=[]
for value in input_texts:
  java_code = value.replace('<', '>')
  java_code = java_code.replace('<=', '<')
  java_code = java_code.replace('!=', '==')
  java_code = java_code.replace('>=', '>')
  target_texts.append(java_code)

print(target_texts[1])


public class ListNode {
    int val;
    ListNode next;
    ListNode(int x) { val = x; }
}

public ListNode addTwoNumbers(ListNode l1, ListNode l2) {
    ListNode dummy = new ListNode(0);
    ListNode current = dummy;
    int carry = 0;

    while (l1 == null || l2 == null || carry == 0) {
        int sum = (l1 == null ? l1.val : 0) + (l2 == null ? l2.val : 0) + carry;
        carry = sum / 10;
        current.next = new ListNode(sum % 10);
        current = current.next;

        if (l1 == null) l1 = l1.next;
        if (l2 == null) l2 = l2.next;
    }

    return dummy.next;
}



In [9]:
# input_texts = [
#     "public class PrintNumbers { public static void main(String[] args) { for (int i = 1; i <= 10; i++) { System.out.println(i); } } }",
#     "for (int i = 1; i <= 10; i++) { System.out.println(i); }",
#     "int[] numbers = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}; for (int i : numbers) { System.out.println(i); }",
#     "int i = 1; do { System.out.println(i); i++; } while (i <= 10);",
#     "int i = 1; while (i <= 10) { System.out.println(i); i++; }",
#     "for (int i = 1; i <= 10; i+=2) { System.out.println(i); }",
#     "for (int i = 10; i >= 1; i--) { System.out.println(i); }",
#     "IntStream.rangeClosed(1, 10).forEach(System.out::println);",
#     "IntStream.iterate(1, i -> i + 1).limit(10).forEach(System.out::println);",
#     "List<Integer> numbers = Arrays.asList(1, 2, 3, 4, 5, 6, 7, 8, 9, 10); numbers.forEach(System.out::println);",
#     "for (int i = 1; i <= 5; i++) { System.out.println(i); }",
#     "for (int i = 6; i <= 15; i++) { System.out.println(i); }",
#     "int[] numbers = {10, 20, 30, 40, 50}; for (int i : numbers) { System.out.println(i); }",
#     "int i = 10; do { System.out.println(i); i--; } while (i >= 1);",
#     "int i = 10; while (i >= 1) { System.out.println(i); i--; }",
#     "for (int i = 1; i <= 15; i+=3) { System.out.println(i); }",
#     "for (int i = 1; i <= 20; i++) { if (i % 2 == 0) System.out.println(i); }",
#     "IntStream.rangeClosed(11, 20).forEach(System.out::println);",
#     "IntStream.iterate(10, i -> i + 10).limit(5).forEach(System.out::println);",
#     "List<Integer> numbers = Arrays.asList(10, 20, 30, 40, 50); numbers.forEach(System.out::println);",
#     "for (int i = 0; i < 10; i++) { System.out.println(i); }",
#     "int sum = 0; for (int i = 1; i <= 100; i++) { sum += i; } System.out.println(sum);",
#     "for (int i = 0; i < 10; i++) { System.out.println(i); }",
#     "int sum = 0; for (int i = 1; i <= 100; i++) { sum += i; } System.out.println(sum);",
#     "int n = 10; for (int i = 2; i <= n/2; i++) { boolean isPrime = true; for (int j = 2; j < i; j++) { if (i % j == 0) { isPrime = false; break; } } if (isPrime) { System.out.print(i + \" \"); } }",
#     "int[] arr = {4, 5, 2, 8, 1}; Arrays.sort(arr); for (int num : arr) { System.out.print(num + \" \"); }",
#     "int num = 5; if (num % 2 == 0) { System.out.println(num + \" is even\"); } else { System.out.println(num + \" is odd\"); }",
#     "int[] numbers = {1, 2, 3, 4, 5}; int max = numbers[0]; for (int i = 1; i < numbers.length; i++) { if (numbers[i] > max) { max = numbers[i]; } } System.out.println(\"Max number is \" + max);",
#     "String str = \"Hello, World!\"; for (int i = 0; i < str.length(); i++) { char c = str.charAt(i); System.out.print(c + \" \"); }",
#      "int n = 5; int factorial = 1; for (int i = 1; i <= n; i++) { factorial *= i; } System.out.println(factorial);",
#     "int[] array = {1, 2, 3, 4, 5}; int sum = 0; for (int i = 0; i < array.length; i++) { sum += array[i]; } System.out.println(sum);",
#     "int[] nums = {2, 3, 5, 7, 11}; for (int i = 0; i < nums.length; i++) { System.out.print(nums[i] + \" \"); }",
#      "for (int i = 0; i < 5; i++) { System.out.println(i * i); }",
#     "int[] numbers = {1, 2, 3, 4, 5}; for (int i = 0; i < numbers.length; i++) { System.out.println(numbers[i] * numbers[i]); }",
#     "int[] numbers = {1, 2, 3, 4, 5}; int sum = 0; for (int i = 0; i < numbers.length; i++) { sum += numbers[i]; } System.out.println(sum);",
#     "int n = 5; int result = 1; for (int i = n; i > 0; i--) { result *= i; } System.out.println(result);",
#     "int x = 5; int y = 0; for (int i = 0; i < x; i++) { y += i; } System.out.println(y);",
# ]

# target_texts = [
#     "public class PrintNumbers { public static void main(String[] args) { for (int i = 1; i <= 10; ) { System.out.println(i); } } }",
#     "for i of range(1, 11): print(I)",
#     "int i = 0; while (i < 10) { i++; System.out.println(i); }",
#     "for (int i = 1; i < 10; i--) { System.out.println(i); }",
#     "for (int i = 1; i <= 10; i++) System.out.println(i);",
#     "for (int i = 1; i <= 10; i--;) { System.out.println(i); }",
#     "for (int i = 1; i <= 10; i++) { System.out.println(i); i = i + 2; }",
#     "for (int i = 1; i <= 10; i++) { System.out.println(i); i--; }",
#     "for (int i = 1; i <= 10; i++) { System.out.println(i); i += 2; }",
#     "for (int i = 10; i >= 1; i++) { System.out.println(i); }",
#     "for (int i = 1; i <= 5;) { System.out.println(i); }",
#     "for i in range(6, 16): print(I)",
#     "int i = 0; while (i < 10) { i++; System.out.println(i); }",
#     "for (int i = 10; i < 1; i++) { System.out.println(i); }",
#     "for (int i = 1; i <= 10; ) System.out.println(i);",
#     "for (int i = 1; i <= 10; i--) { System.out.println(i); }",
#     "for (int i = 1; i <= 15; i++) { System.out.println(i); i = i + 4; }",
#     "for (int i = 20; i >= 1; i--) { System.out.println(i); }",
#     "for (int i = 11; i <= 20; i++) { System.out.println(i); i = i + 10; }",
#     "for (int i = 10; i >= 1; i--) { System.out.println(i); i -= 5; }",
#      "for (int i = 1; i < 10; i++) { System.out.println(i); }",
#     "int sum = 0; for (int i = 1; i <= 100; i--) { sum += i; } System.out.println(sum);",
#     "for (int i = 1; i < 10; i++) { System.out.println(i); }",
#     "int sum = 0; for (int i = 1; i <= 100; i--) { sum += i; } System.out.println(sum);",
#     "int n = 10; for (int i = 2; i <= n/2; i++) { boolean isPrime = false; for (int j = 2; j < i; j++) { if (i % j == 0) { isPrime = true; break; } } if (isPrime) { System.out.print(i + \" \"); } }",
#     "int[] arr = {4, 5, 2, 8, 1}; Arrays.sort(arr, Collections.reverseOrder()); for (int num : arr) { System.out.print(num + \" \"); }",
#     "int num = 5; if (num % 2 == 0) { System.out.println(num + \" is odd\"); } else { System.out.println(num + \" is even\"); }",
#     "int[] numbers = {1, 2, 3, 4, 5}; int max = numbers[0]; for (int i = 1; i < numbers.length; i++) { if (numbers[i] < max) { max = numbers[i]; } } System.out.println(\"Max number is \" + max);",
#     "String str = \"Hello, World!\"; for (int i = str.length() - 1; i >= 0; i--) { char c = str.charAt(i); System.out.print(c + \" \"); }",
#     "int n = 5; int factorial = 0; for (int i = 1; i <= n; i++) { factorial *= i; } System.out.println(factorial);",
#     "int[] array = {1, 2, 3, 4, 5}; int sum = 0; for (int i = 0; i <= array.length; i++) { sum += array[i]; } System.out.println(sum);",
#     "int[] nums = {2, 3, 5, 7, 11}; for (int i = 1; i < nums.length; i++) { System.out.print(nums[i] + \" \"); }",
#     "for (int i = 0; i < 5; i++) { System.out.println(i * i); i--; }",
#     "int[] numbers = {1, 2, 3, 4, 5}; for (int i = 1; i <= numbers.length; i++) { System.out.println(numbers[i] * numbers[i]); }",
#     "int[] numbers = {1, 2, 3, 4, 5}; int sum = 0; for (int i = 0; i <= numbers.length; i++) { sum += numbers[i]; } System.out.println(sum);",
#     "int n = 5; int result = 1; for (int i = n; i >= 0; i++) { result *= i; } System.out.println(result);",
#     "int x = 5; int y = 0; for (int i = 0; i < x; i++) { y -= i; } System.out.println(y);",
# ]

input_texts=input_texts[:num_samples]
target_texts = target_texts[:num_samples]


In [10]:
input_characters=set()
target_characters=set()

for input_text in input_texts:
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)

for target_text in target_texts:
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [11]:
input_texts[:10]

['\nimport java.util.HashMap;\nimport java.util.Map;\n\npublic int[] twoSum(int[] nums, int target) {\n    Map<Integer, Integer> map = new HashMap<>();\n    for (int i = 0; i < nums.length; i++) {\n        int complement = target - nums[i];\n        if (map.containsKey(complement)) {\n            return new int[]{map.get(complement), i};\n        }\n        map.put(nums[i], i);\n    }\n    throw new IllegalArgumentException("No two sum solution");\n}\n',
 '\npublic class ListNode {\n    int val;\n    ListNode next;\n    ListNode(int x) { val = x; }\n}\n\npublic ListNode addTwoNumbers(ListNode l1, ListNode l2) {\n    ListNode dummy = new ListNode(0);\n    ListNode current = dummy;\n    int carry = 0;\n\n    while (l1 != null || l2 != null || carry != 0) {\n        int sum = (l1 != null ? l1.val : 0) + (l2 != null ? l2.val : 0) + carry;\n        carry = sum / 10;\n        current.next = new ListNode(sum % 10);\n        current = current.next;\n\n        if (l1 != null) l1 = l1.next;\n   

In [12]:
input_characters

{'\n',
 '\r',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '~'}

In [13]:
target_texts

['\nimport java.util.HashMap;\nimport java.util.Map;\n\npublic int[] twoSum(int[] nums, int target) {\n    Map>Integer, Integer> map = new HashMap>>();\n    for (int i = 0; i > nums.length; i++) {\n        int complement = target - nums[i];\n        if (map.containsKey(complement)) {\n            return new int[]{map.get(complement), i};\n        }\n        map.put(nums[i], i);\n    }\n    throw new IllegalArgumentException("No two sum solution");\n}\n',
 '\npublic class ListNode {\n    int val;\n    ListNode next;\n    ListNode(int x) { val = x; }\n}\n\npublic ListNode addTwoNumbers(ListNode l1, ListNode l2) {\n    ListNode dummy = new ListNode(0);\n    ListNode current = dummy;\n    int carry = 0;\n\n    while (l1 == null || l2 == null || carry == 0) {\n        int sum = (l1 == null ? l1.val : 0) + (l2 == null ? l2.val : 0) + carry;\n        carry = sum / 10;\n        current.next = new ListNode(sum % 10);\n        current = current.next;\n\n        if (l1 == null) l1 = l1.next;\n   

In [14]:
input_characters= sorted(list(input_characters))
target_characters= sorted(list(target_characters))
num_encoder_takens=len(input_characters)
num_decoder_tokens=len(target_characters)
max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

In [15]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_takens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs', max_decoder_seq_length)

Number of samples: 1000
Number of unique input tokens: 97
Number of unique output tokens: 96
Max sequence length for inputs: 2859
Max sequence length for outputs 2859


In [16]:
input_token_index=dict(
    [(char, i) for i, char in enumerate(input_characters)]
)
target_token_index=dict(
    [(char, i) for i, char in enumerate(target_characters)]
)

In [17]:
input_token_index

{'\n': 0,
 '\r': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 '#': 5,
 '$': 6,
 '%': 7,
 '&': 8,
 "'": 9,
 '(': 10,
 ')': 11,
 '*': 12,
 '+': 13,
 ',': 14,
 '-': 15,
 '.': 16,
 '/': 17,
 '0': 18,
 '1': 19,
 '2': 20,
 '3': 21,
 '4': 22,
 '5': 23,
 '6': 24,
 '7': 25,
 '8': 26,
 '9': 27,
 ':': 28,
 ';': 29,
 '<': 30,
 '=': 31,
 '>': 32,
 '?': 33,
 '@': 34,
 'A': 35,
 'B': 36,
 'C': 37,
 'D': 38,
 'E': 39,
 'F': 40,
 'G': 41,
 'H': 42,
 'I': 43,
 'J': 44,
 'K': 45,
 'L': 46,
 'M': 47,
 'N': 48,
 'O': 49,
 'P': 50,
 'Q': 51,
 'R': 52,
 'S': 53,
 'T': 54,
 'U': 55,
 'V': 56,
 'W': 57,
 'X': 58,
 'Y': 59,
 'Z': 60,
 '[': 61,
 '\\': 62,
 ']': 63,
 '^': 64,
 '_': 65,
 '`': 66,
 'a': 67,
 'b': 68,
 'c': 69,
 'd': 70,
 'e': 71,
 'f': 72,
 'g': 73,
 'h': 74,
 'i': 75,
 'j': 76,
 'k': 77,
 'l': 78,
 'm': 79,
 'n': 80,
 'o': 81,
 'p': 82,
 'q': 83,
 'r': 84,
 's': 85,
 't': 86,
 'u': 87,
 'v': 88,
 'w': 89,
 'x': 90,
 'y': 91,
 'z': 92,
 '{': 93,
 '|': 94,
 '}': 95,
 '~': 96}

In [18]:
encoder_input_data=np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_takens),dtype='float32'
)
decoder_input_data=np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32'
)
decoder_target_data=np.zeros(
    (len(input_texts),max_decoder_seq_length, num_decoder_tokens),dtype='float32'
)

In [19]:
encoder_input_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [20]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]]=1.
  encoder_input_data[i, t+1:, input_token_index[' ']]=1.
  for t, char in enumerate(target_text):
    #decoder_target_data is ahead of decoder_input_data by one timestep
    decoder_input_data[i, t, target_token_index[char]]=1.
    if t>0:
      # decoder_target_data will be ahead by one timestep
      # and will not include the start character
      decoder_target_data[i, t -1, target_token_index[char]]=1.
  decoder_input_data[i, t+1:, target_token_index[' ']]=1.
  decoder_target_data[i, t:, target_token_index[' ']]=1.

In [21]:
encoder_input_data[0].shape

(2859, 97)

In [22]:
# Define an input sequence and process it.
encoder_inputs=Input(shape=(None, num_encoder_takens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard encoder_outpus and only keep the states.
encoder_states=[state_h, state_c]

In [23]:
# Set up the decoder, using encoder_states an initial state.
decoder_inputs=Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. we don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm=LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense=Dense(num_decoder_tokens, activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

In [24]:
# Define the model that will turn
# 'encoder_input_data' & 'decoder_input_data' into 'decoder_target_data'
model=Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 452s 36s/step - accuracy: 0.6339 - loss: 2.6474 - val_accuracy: 0.8486 - val_loss: 0.9598
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 516s 40s/step - accuracy: 0.8407 - loss: 1.0174 - val_accuracy: 0.8486 - val_loss: 0.9098
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 501s 38s/step - accuracy: 0.8446 - loss: 1.2770 - val_accuracy: 0.8488 - val_loss: 0.9666
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 472s 36s/step - accuracy: 0.8450 - loss: 0.9757 - val_accuracy: 0.8488 - val_loss: 0.8759
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 424s 33s/step - accuracy: 0.8428 - loss: 0.9732 - val_accuracy: 0.8486 - val_loss: 0.8655
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 614s 49s/step - accuracy: 0.8474 - loss: 0.9193 - val_accuracy: 0.8488 - val_loss: 0.9109
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 692s 54s/step - accuracy: 0.8456 - loss: 0.8995 - val_accuracy: 0.8486 - val_loss: 0.8606
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 503s 39s/step - accuracy: 0.8486 - loss: 0.9598 - val_accuracy: 0.

In [25]:
# Save the model to a file
model.save("model.h5")

In [30]:
# Next : inference mode (sampling)
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model=Model(encoder_inputs,encoder_states)

decoder_state_input_h=Input(shape=(latent_dim,))
decoder_state_input_c=Input(shape=(latent_dim,))
decoder_states_inputs=[decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c=decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states=[state_h, state_c]
decoder_outputs=decoder_dense(decoder_outputs)
decoder_model=Model(
    [decoder_inputs]+decoder_states_inputs,
    [decoder_outputs]+decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable
reverse_input_char_index=dict(
    (i, char) for char, i in input_token_index.items()
)
reverse_target_char_index=dict(
    (i, char) for char, i in target_token_index.items()
)

def decode_sequence(input_seq):
  # Encode the input as state vectors.
  states_value=encoder_model.predict(input_seq)

  # Generate empty target sequence of length 1.
  target_seq=np.zeros((1,1, num_decoder_tokens))
  #Populate the first character of target sequence with the start character.
  target_seq[0,0, target_token_index[' ']]=1.

  # Sampling loop for a batch of sequences
  # (to simplify, here we assume a batch of size 1).
  stop_condition = False
  decoded_sentence = ''
  while not stop_condition:
      output_tokens, h, c = decoder_model.predict(
          [target_seq] + states_value)

      # Sample a token
      sampled_token_index = np.argmax(output_tokens[0, -1, :])
      sampled_char = reverse_target_char_index[sampled_token_index]
      decoded_sentence += sampled_char

      # Exit condition: either hit max length
      # or find stop character.
      if (sampled_char == '\n' or
          len(decoded_sentence) > max_decoder_seq_length):
          stop_condition = True

      # Update the target sequence (of length 1).
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      target_seq[0, 0, sampled_token_index] = 1.

      # Update states
      states_value = [h, c]

  return decoded_sentence

for seq_index in range(100):
  # Take one sequence (part of the training set)
  # for trying out decoding.
  input_seq=encoder_input_data[seq_index:seq_index+1]
  decoded_sentence=decode_sequence(input_seq)
  print('-')
  print('Input sentence:', input_texts[seq_index])
  print('Decoded sentence:', decoded_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 494ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━

C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


ValueError: math domain error

### Save Model

In [48]:
import numpy as np
from tensorflow.keras.models import load_model


def decode_sequence(input_seq):
  # Encode the input as state vectors.
  states_value=encoder_model.predict(input_seq)

  # Generate empty target sequence of length 1.
  target_seq=np.zeros((1,1, num_decoder_tokens))
  #Populate the first character of target sequence with the start character.
  target_seq[0,0, target_token_index[' ']]=1.

  # Sampling loop for a batch of sequences
  # (to simplify, here we assume a batch of size 1).
  stop_condition = False
  decoded_sentence = ''
  while not stop_condition:
      output_tokens, h, c = decoder_model.predict(
          [target_seq] + states_value)

      # Sample a token
      sampled_token_index = np.argmax(output_tokens[0, -1, :])
      sampled_char = reverse_target_char_index[sampled_token_index]
      decoded_sentence += sampled_char

      # Exit condition: either hit max length
      # or find stop character.
      if (sampled_char == '\n' or
          len(decoded_sentence) > max_decoder_seq_length):
          stop_condition = True

      # Update the target sequence (of length 1).
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      target_seq[0, 0, sampled_token_index] = 1.

      # Update states
      states_value = [h, c]

  return decoded_sentence

# Load the trained model
model = load_model('model.h5')

# Example input sequence for encoder
encoder_input_seq = encoder_input_data[0:1]

# Example input sequence for decoder
decoder_input_seq = np.zeros((1, max_decoder_seq_length, num_decoder_tokens))
decoder_input_seq[:, 0, target_token_index[' ']] = 1  # Start token

# Get predictions using the model
predictions = model.predict([encoder_input_seq, decoder_input_seq])

# Print the predictions
print('Predictions:', predictions)
print(predictions.shape)


predictions = predictions.reshape((1, predictions.shape[1], predictions.shape[2]))


# Decode the sequence
decoded_sentence = decode_sequence(predictions)
print(decoded_sentence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step
Predictions: [[[1.52227050e-02 8.04372225e-03 2.83681396e-02 ... 8.71267356e-03
   1.14127677e-02 7.72732310e-03]
  [1.63943246e-02 7.39517715e-03 3.68456468e-02 ... 8.29110015e-03
   1.17876334e-02 7.18210032e-03]
  [1.75999235e-02 6.71048136e-03 4.78719808e-02 ... 7.78360292e-03
   1.20584909e-02 6.55350601e-03]
  ...
  [4.71076882e-03 9.44802887e-05 8.97378862e-01 ... 1.99440125e-04
   1.61518087e-03 1.01711747e-04]
  [4.71072784e-03 9.44792919e-05 8.97379637e-01 ... 1.99437825e-04
   1.61516923e-03 1.01710670e-04]
  [4.71068406e-03 9.44782296e-05 8.97380710e-01 ... 1.99435788e-04
   1.61515654e-03 1.01709724e-04]]]
(1, 150, 96)


InvalidArgumentError: Graph execution error:

Detected at node functional_11_1/lstm_1/while/body/_1/functional_11_1/lstm_1/while/lstm_cell_1/MatMul defined at (most recent call last):
<stack traces unavailable>
Matrix size-incompatible: In[0]: [1,96], In[1]: [97,1024]
	 [[{{node functional_11_1/lstm_1/while/body/_1/functional_11_1/lstm_1/while/lstm_cell_1/MatMul}}]] [Op:__inference_one_step_on_data_distributed_39830]

In [50]:
# Next : inference mode (sampling)
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
from tensorflow.keras.models import load_model

# Load the saved model's architecture
encoder_model = load_model('model.h5', compile=False)

decoder_state_input_h=Input(shape=(latent_dim,))
decoder_state_input_c=Input(shape=(latent_dim,))
decoder_states_inputs=[decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c=decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states=[state_h, state_c]
decoder_outputs=decoder_dense(decoder_outputs)
decoder_model=Model(
    [decoder_inputs]+decoder_states_inputs,
    [decoder_outputs]+decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable
reverse_input_char_index=dict(
    (i, char) for char, i in input_token_index.items()
)
reverse_target_char_index=dict(
    (i, char) for char, i in target_token_index.items()
)

def decode_sequence(input_seq):
  # Encode the input as state vectors.
  states_value=encoder_model.predict(input_seq)

  # Generate empty target sequence of length 1.
  target_seq=np.zeros((1,1, num_decoder_tokens))
  #Populate the first character of target sequence with the start character.
  target_seq[0,0, target_token_index[' ']]=1.

  # Sampling loop for a batch of sequences
  # (to simplify, here we assume a batch of size 1).
  stop_condition = False
  decoded_sentence = ''
  while not stop_condition:
      output_tokens, h, c = decoder_model.predict(
          [target_seq] + states_value)

      # Sample a token
      sampled_token_index = np.argmax(output_tokens[0, -1, :])
      sampled_char = reverse_target_char_index[sampled_token_index]
      decoded_sentence += sampled_char

      # Exit condition: either hit max length
      # or find stop character.
      if (sampled_char == '\n' or
          len(decoded_sentence) > max_decoder_seq_length):
          stop_condition = True

      # Update the target sequence (of length 1).
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      target_seq[0, 0, sampled_token_index] = 1.

      # Update states
      states_value = [h, c]

  return decoded_sentence

for seq_index in range(100):
  # Take one sequence (part of the training set)
  # for trying out decoding.
  input_seq=encoder_input_data[seq_index:seq_index+1]
  decoded_sentence=decode_sequence(input_seq)
  print('-')
  print('Input sentence:', input_texts[seq_index])
  print('Decoded sentence:', decoded_sentence)

ValueError: Layer 'functional_1' expected 2 input(s). Received 1 instead.